# CLIMADA

In [ ]:
import time

import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm
from datetime import date

from climada.hazard import TropCyclone
from climada.entity import Exposures
from climada.entity.exposures import LitPop
from climada.entity.impact_funcs import ImpactFuncSet, ImpfTropCyclone

import adjusted_year_loss_tables as aylt

In [ ]:
## set parameters
# Exposure
my_country = 'aus'
m, n = 1,1 # based on analysis by Eberenz et al. (2020), (1,1) is the best combination.
my_fin_mode = 'pc'
ref_year = 2023

# Hazard
start_year = 1980
end_year = 2023
synthetic_start_year = end_year + 1
n_synth_tracks = 1_000

my_res_arcsec = 600
starting_phase = 'Nino'

## Exposure

In [ ]:
# Exposure
exp1 = LitPop.from_countries(my_country, my_res_arcsec, (m,n), my_fin_mode, reference_year = ref_year)
exp1.check()

# Remove Lord Howe Island and Macquarie Island
exp = Exposures(exp1.gdf[exp1.geometry.x <= 155])
exp.check()

In [ ]:
exp_with_state = aylt.assign_state_to_exposure(exp.data, "STE_2021_AUST_SHP_GDA2020/STE_2021_AUST_GDA2020.shp").replace("Australian Capital Territory", "New South Wales")

In [ ]:
regions = exp_with_state["STE_NAME21"]
regions

In [ ]:
state_exposures = exp_with_state.groupby("STE_NAME21")["value"].sum()
region_thresholds = dict(state_exposures * 1e-6)
region_thresholds

In [ ]:
for region in region_thresholds:
    print(f"{region} threshold: ${region_thresholds[region]:.4} USD")

In [ ]:
# check
print(f'Total exposure value to {my_country}: USD {exp.gdf.value.sum():,.2f} / AUD {exp.gdf.value.sum()/0.6630:,.2f} ({ref_year})')

## Hazard

In [ ]:
# Hazard
haz = TropCyclone.from_hdf5(f'Hazards/haz_aus_{n_synth_tracks}synth.hdf5')
haz.check()
exp.assign_centroids(haz)

## Impact

In [ ]:
impf_tc = ImpfTropCyclone.from_emanuel_usa()
impf_set = ImpactFuncSet([impf_tc])
impf_set.check()

In [ ]:
loss_catalogues = aylt.compute_loss_catalogue(haz, exp, impf_set, save_catalogue=True)

In [ ]:
def resample_losses_timed(kwargs):
    start_time = time.time()
    base_filename = aylt.generate_year_loss_tables(**kwargs)
    print(f"Finished {base_filename} with in {time.time() - start_time:.2f} seconds")
    return base_filename

In [ ]:
n_sim = 1_000_000

In [ ]:
loss_configs = [
    dict(homogeneous=True, loc=False, intensity=False, damage=False),
    dict(homogeneous=False, loc=False, intensity=False, damage=False),
    dict(homogeneous=False, loc=True, intensity=True, damage=True, p_loc=0.5),
    dict(homogeneous=False, loc=True, intensity=True, damage=True, p_loc=0.8),
    dict(homogeneous=False, loc=True, intensity=True, damage=True, p_loc=1.0),
    dict(homogeneous=False, loc=False, intensity=True, damage=False),
    dict(homogeneous=False, loc=False, intensity=False, damage=True),
    dict(homogeneous=False, loc=True, intensity=False, damage=False, p_loc=0.5),
    dict(homogeneous=False, loc=True, intensity=True, damage=False, p_loc=0.5),
    dict(homogeneous=False, loc=True, intensity=False, damage=True, p_loc=0.5),
    dict(homogeneous=False, loc=False, intensity=True, damage=True),
]

# Make a random shared seed to use for each simulation
rng = np.random.default_rng(2025)
seed = rng.integers(0, 2**32, dtype=np.uint32)

for i, cfg in enumerate(loss_configs):
    cfg["haz"] = haz
    cfg["n_sim"] = n_sim
    cfg["n_years"] = 1
    cfg["loss_catalogues"] = loss_catalogues
    cfg["starting_enso"] = starting_phase
    cfg["seed"] = seed
    cfg["regions"] = regions
    cfg["region_thresholds"] = region_thresholds
    cfg["synthetic_start_year"] = synthetic_start_year
    cfg["compound_factor"] = 1.5
    cfg["show_progress"] = True
    cfg["save_poisson_debug"] = True
    cfg["save_damage_debug"] = True

In [ ]:
%%time
results1 = Parallel(n_jobs=10)(
    delayed(resample_losses_timed)(cfg)
    for cfg in tqdm(loss_configs, desc="Running simulations")
)

In [ ]:
%%time
# Make a copy of the loss_configs list but change the n_year to 5 and make a new seed
rng = np.random.default_rng(2026)
seed = rng.integers(0, 2**32, dtype=np.uint32)

loss_configs_5 = [cfg.copy() for cfg in loss_configs]
for i, cfg in enumerate(loss_configs_5):
    cfg["n_years"] = 5
    cfg["seed"] = seed
    cfg["show_progress"] = True

results5 = Parallel(n_jobs=10)(
    delayed(resample_losses_timed)(cfg)
    for cfg in tqdm(loss_configs_5, desc="Running simulations")
)